In [1]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Musae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Musae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Define Arabic and English punctuation
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

# Define Arabic stopwords
arabic_stopwords = set(stopwords.words('arabic'))

In [3]:
# Define Arabic stopwords
arabic_stopwords = set(stopwords.words('arabic'))

# Function to normalize Arabic script
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)  # Corrected this line
    text = re.sub("گ", "ك", text)
    return text


# Function to process and stem Arabic text using ISRI stemmer
def process_text(text):
    # Normalize Arabic text
    text = normalize_arabic(text)
    # Tokenize text
    word_list = word_tokenize(text)
    # Initialize the ISRI stemmer
    stemmer = ISRIStemmer()
    # Apply stemming to each word
    word_list = [stemmer.stem(w) for w in word_list if w not in punctuations_list and w not in arabic_stopwords]
    return ' '.join(word_list)

In [4]:
# Load the dataset
df = pd.read_csv('C:\\Users\\Musae\\Documents\\GitHub-REPOs\\NLP-Project\\data\\ar_reviews_100k.tsv', sep='\t')

# Apply the new processing function to the text column
df['cleaned_text'] = df['text'].apply(process_text)

# Display the original and processed text for the first 10 entries
print(df[['text', 'cleaned_text']].head(10))

                                                text  \
0  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...   
1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...   
2  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...   
3  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...   
4  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...   
5  أسلوب الكاتب رائع جدا و عميق جدا، قرأته عدة مر...   
6  استثنائي. الهدوء في الجناح مع مسبح. عدم وجود ع...   
7  الكتاب هو السيرة الذاتية للحداثة في المملكة بل...   
8       من أجمل ما قرأت.. رواية تستحق القراءة فعلا..   
9  بشكل عام جيده .. . التجاوب جيد جدا من قبل موظف...   

                                        cleaned_text  
0                      متز نوع نظف وقع جهز شاطيء طعم  
1  احد سبب نجح امر ان شخص دول عشق ترب نحب امر ومض...  
2  هدف .. وقي نقل صخب شرع قهر الي هدء جبل شيش .. ...  
3  خلص .. بدي الل مست بهر زي فيل زرق ميقراش احس ....  
4       ياس جلر جزء تجز دبي ندق كامل خدم ريح نفس وجد  
5  سلب كتب ريع جدا عمق جد، قرت عده رات كنت طلب رح... 

In [5]:
positive_keywords = {
    'ممتاز', 'جيد', 'رائع', 'سعيد', 'لذيذ', 'مبهج', 'فرح', 'استثنائي', 'جميل', 'محبب', 'ممتع',
    'مذهل', 'مريح', 'راض', 'أحب', 'استمتع', 'مفاجأة', 'مميز', 'لطيف', 'مرح', 'معجزة',
    'ملهم', 'أسعد', 'خيالي', 'مذهل', 'فريد', 'هائل', 'راقي', 'أنيق', 'بهجة', 'مفيد',
    'قيمة', 'بسيط', 'ناجح', 'موفق', 'مشجع', 'حسن', 'ظريف', 'محبوب', 'مبهر', 'إيجابي',
    'تفاؤل', 'إعجاب', 'ممتن', 'شجاع', 'آمن', 'مثالي'
}

negative_keywords = {
    'سيء', 'مخيب', 'حزين', 'مؤلم', 'كريه', 'قبيح', 'فشل', 'محبط', 'بشع', 'فظيع', 'مزعج', 
    'مروع', 'أسوأ', 'كره', 'كارثة', 'رعب', 'كئيب', 'مزعزع', 'اكتئاب', 'بائس', 'معقد', 
    'إحباط', 'تعب', 'مضجر', 'ممل', 'فضيحة', 'سلبي', 'كاذب', 'فظاظة', 'احتيال', 'احراج',
    'بشع', 'تعيس', 'مستاء', 'مروع', 'مشؤوم', 'عداء', 'مزري', 'عنيف', 'غير كاف', 'غير صادق',
    'ضعيف', 'متشائم', 'غاضب', 'غير مقبول', 'مروع'
}


In [6]:
# Rule-based classification function
def classify_review(text):
    # Tokenize the input text
    words = set(text.split())
    # Initialize flags to check for keyword presence
    has_positive = bool(words & positive_keywords)
    has_negative = bool(words & negative_keywords)

    # Basic rule-based classification
    if has_positive and not has_negative:
        return "Positive"
    elif has_negative and not has_positive:
        return "Negative"
    else:
        return "Mixed"

# Apply classification to cleaned text
df['predicted_label'] = df['cleaned_text'].apply(classify_review)

# Display the original, cleaned, and predicted labels for the first 10 reviews
print(df[['text', 'cleaned_text', 'label', 'predicted_label']].head(10))

# Evaluate the classifier
predicted_counts = df['predicted_label'].value_counts()

print("\nPredicted sentiment counts:")
print(predicted_counts)


                                                text  \
0  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...   
1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...   
2  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...   
3  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...   
4  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...   
5  أسلوب الكاتب رائع جدا و عميق جدا، قرأته عدة مر...   
6  استثنائي. الهدوء في الجناح مع مسبح. عدم وجود ع...   
7  الكتاب هو السيرة الذاتية للحداثة في المملكة بل...   
8       من أجمل ما قرأت.. رواية تستحق القراءة فعلا..   
9  بشكل عام جيده .. . التجاوب جيد جدا من قبل موظف...   

                                        cleaned_text     label predicted_label  
0                      متز نوع نظف وقع جهز شاطيء طعم  Positive           Mixed  
1  احد سبب نجح امر ان شخص دول عشق ترب نحب امر ومض...  Positive           Mixed  
2  هدف .. وقي نقل صخب شرع قهر الي هدء جبل شيش .. ...  Positive           Mixed  
3  خلص .. بدي الل مست بهر زي فيل زرق ميقراش احس ....  Posit